### Logistic Regression
- Estimates the probability of an event occurring based on independent variables
- Models a binary outcome (True/False, Yes/No)
    - In this example, the outcome is either win or loss
- Equation: $$ f(x) = {e^{b} \over (1+e^{-x})} $$

### Prerequisite:
- Download Anaconda (https://www.anaconda.com/download)
- Install Github (https://gitforwindows.org/) (optional)
- Set up Github account (optional)
- Clone repository (git clone https://github.com/Krescenskok/DataScienceSessions.git) (optional)
- Useful git commands (https://git-scm.com/docs)

### Problem Statement and Hypothesis
- Research Question: Can the business predict opportunity outcomes?
- Alternate Hypothesis: Yes, we can predict opportunity outcomes
- Null Hypothesis: No, we cannot predict opportunity outcomes

### Why is it important to predict an opportunity with statistical significance?
- Anticipate sales for CST and other leadership
- Help influence a positive behavior


**Expected Outcome:** To have a model that can generalize trends in data using significant variables to predict a win or a loss.
- Understand which opportunity variables are most significant
- Figure out the accuracy of the model

**Data Collection Concerns:**
- Dataset needs to have at least 10,000 records to prevent bias
- At least 10 variables 
- At least 1 continuous and 1 categorical variable
- Rights to use the data

**Data Variables:**
- **Independent (x<sub>1</sub>-x<sub>n</sub>):** All other variables
- **Dependent (Y):** Stage (Categorical variable - Outcomes are only win or loss)
    - Logistic regressions are used to predict categorical outcomes

**Packages:**
- **pandas:** Used for dataframes
- **pyodbc:** Used to connect to SQL Server
- **sklearn/stats model API:** Used for our logistic model
    - Create train and test sets
    - Recursive feature elimination
    - Accuracy/precision/matrix scores
    - Receiver operator characteristic
- **seaborn/matplotlib:** Used for visualization

**Preprocessing Technique:**
- Import packages and load data
- Review the data
    - head(), info()
    - Handle missingness, outliers
- Exploratory analysis
- Variable selection

### Import Packages and load data

In [78]:
#import packages
import pyodbc as db
import pandas as pd

#connect to DB
server = 'tcp:usaepwvds025.database.windows.net' 
database = 'SIMS' 
username = 'ciaserviceaccount' 
password = 'pipeline$330' 
cnxn = db.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

#load dataframe
sql = "select * from sims.logisticRegression"
df = pd.read_sql(sql,cnxn)


C:\Users\EHIDRPC\AppData\Local\Temp\ipykernel_52144\1958964111.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,cnxn)


### Review the data
- Understand data types (Continuous vs Categorical)
- Investigate missingness
- Scale
- Outliers

In [79]:
df.head()

LegacySFInstance     LegacyAccountID  \
0              RxC  0013s00000uY8t8AAC   
1              RxC  0013s00000xIwQOAA0   
2              RxC  0013s00000zTk3cAAC   
3              RxC  0013s000011ZefwAAC   
4              RxC  0013s000011ZefwAAC   

                          LegacyAccountName  \
0  Coram Speciality Infusion Services, Inc.   
1                                 Caris-MPI   
2                                Pharma Pac   
3        Biologics Financial Reconciliation   
4        Biologics Financial Reconciliation   

                        CurrentCompanyGroup  CurrentCompanyID  \
0  Coram Speciality Infusion Services, Inc.               NaN   
1                                 Caris-MPI               NaN   
2                                Pharma Pac               NaN   
3        Biologics Financial Reconciliation               NaN   
4        Biologics Financial Reconciliation               NaN   

   CurrentCompanyMarketCap CurrentCompanyMarketCapCategory Top12CMMCompany  \
0                      NaN                            None            None   
1                      NaN                            None            None   
2                      NaN                            None            None   
3                      NaN                            None            None   
4                      NaN                            None            None   

  Top24CMMCompany       OpportunityID  \
0            None  0063s00000DSJH9AAP   
1            None  0063s00000EAht5AAD   
2            None  0063s00000DUuUEAA1   
3            None  0063s00000EwxCLAAZ   
4            None  0063s00000ExCSbAAN   

                               OpportunityName OpportunityType  \
0                       Coram/CVS Copay Finder             New   
1                    Precision Medicine Survey             New   
2                                   Pharma Pac             New   
3  Biologics RARE - Launched for FY22 Tracking         Renewal   
4   Biologics ONC - Launched for FY22 Tracking         Renewal   

   TotalOpportunityAmount          CreateDate   StartDate OpportunityOwner  \
0                     NaN 2019-08-06 19:15:55        None    Andrew Burman   
1                200000.0 2017-05-31 15:15:14        None  Jennifer Frytak   
2                173838.0 2020-04-09 15:52:13  2021-04-12    Ryan Grimmett   
3              44986789.0 2021-02-04 14:55:41  2021-04-01      Laura Sieve   
4              18086000.0 2021-04-27 15:01:37  2021-04-01      Laura Sieve   

     StageName         CloseReason                          CloseComments  \
0  Closed Lost              No Bid  Nothing materialized with the client.   
1  Closed Lost  Client Funding Cut                                   None   
2  Closed Lost           Cancelled                                   None   
3   Closed Won                None                                   None   
4   Closed Won                None                                   None   

  ImpactedFY  
0    FY 2020  
1    FY 2018  
2    FY 2022  
3    FY 2022  
4    FY 2022

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27528 entries, 0 to 27527
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   LegacySFInstance                 27528 non-null  object        
 1   LegacyAccountName                27508 non-null  object        
 2   CurrentCompanyGroup              27508 non-null  object        
 3   CurrentCompanyMarketCap          23076 non-null  float64       
 4   CurrentCompanyMarketCapCategory  27451 non-null  object        
 5   Top12CMMCompany                  10023 non-null  object        
 6   Top24CMMCompany                  4450 non-null   object        
 7   OpportunityName                  27528 non-null  object        
 8   OpportunityType                  27528 non-null  object        
 9   TotalOpportunityAmount           26194 non-null  float64       
 10  CreateDate                       27528 non-null  datetime6

#### Drop ID Columns
- Looking for patterns in the data. Since the ID columns are unique, this won't help us in predicting our win/loss opportunities

In [80]:
df = df.drop(columns=['LegacyAccountID', 'CurrentCompanyID', 'OpportunityID'])

We can't impute comments, so we will drop `CloseComments` from our dataset

In [81]:
df = df.drop(columns = 'CloseComments')

Since we have `CreateDate` that have no null values, we will use this column and drop `StartDate`

In [82]:
df = df.drop(columns = 'StartDate')

In [83]:
df.query('LegacyAccountName != LegacyAccountName')

LegacySFInstance LegacyAccountName CurrentCompanyGroup  \
57              CMM              None                None   
58              CMM              None                None   
59              CMM              None                None   
60              CMM              None                None   
61              CMM              None                None   
62              CMM              None                None   
63              CMM              None                None   
64              CMM              None                None   
65              CMM              None                None   
66              CMM              None                None   
67              CMM              None                None   
68              CMM              None                None   
69              CMM              None                None   
70              RHP              None                None   
71              RHP              None                None   
72              RHP              None                None   
73              RHP              None                None   
74              RHP              None                None   
75              RHP              None                None   
76              RHP              None                None   

    CurrentCompanyMarketCap CurrentCompanyMarketCapCategory Top12CMMCompany  \
57                      NaN                            None            None   
58                      NaN                            None            None   
59                      NaN                            None            None   
60                      NaN                            None            None   
61                      NaN                            None            None   
62                      NaN                            None            None   
63                      NaN                            None            None   
64                      NaN                            None            None   
65                      NaN                            None            None   
66                      NaN                            None            None   
67                      NaN                            None            None   
68                      NaN                            None            None   
69                      NaN                            None            None   
70                      NaN                            None            None   
71                      NaN                            None            None   
72                      NaN                            None            None   
73                      NaN                            None            None   
74                      NaN                            None            None   
75                      NaN                            None            None   
76                      NaN                            None            None   

   Top24CMMCompany                                    OpportunityName  \
57            None  MDD Xadago (Safinamide) Prior Authorization Pl...   
58            None  MDD Xadago (Safinamide) Prior Authorization Pl...   
59            None                 MDD Xadago (Safinamide) HubExpress   
60            None  MDD Xadago (Safinamide) Prior Authorization Pl...   
61            None  testpharma Brand 1 Test Prior Authorization Pl...   
62            None                              testpharma Brand4 IAN   
63            None  testpharma Brand 1 Test Provider Medication Al...   
64            None       testpharma BrandABC Prior Authorization Plus   
65            None  testpharma BB TEST BRAND Prior Authorization Plus   
66            None  testpharma BB TEST BRAND Prior Authorization Plus   
67            None   testpharma Multibrand Clinical Alerts - Vaccines   
68            None     testpharma Multibrand Prior Authorization Plus   
69            None                testpharma BB TEST BRAND HubExpress   
70            None             Kentucky

In [84]:
df.dropna(subset=['LegacyAccountName'], inplace=True)

Impute a value for missing market caps
- Average value in the market cap category
- Some companies don't have a market cap category, so we will first impute market cap categories

In [85]:
modeMarketCap = df['CurrentCompanyMarketCapCategory'].mode()
print(modeMarketCap)
df['CurrentCompanyMarketCapCategory'].fillna(df['CurrentCompanyMarketCapCategory'].mode()[0], inplace=True)

0    Large
Name: CurrentCompanyMarketCapCategory, dtype: object


In [86]:
meanMarketCap = df.groupby('CurrentCompanyMarketCapCategory')['CurrentCompanyMarketCap'].mean().reset_index()
print(meanMarketCap)
totalMeanMarketCap = df['CurrentCompanyMarketCap'].mean()
print(totalMeanMarketCap)

  CurrentCompanyMarketCapCategory  CurrentCompanyMarketCap
0                           Large             8.923380e+10
1                          Medium             5.180624e+09
2                            Mega             3.616713e+11
3                           Micro             8.615721e+07
4                   Not Available                      NaN
5                         Private                      NaN
6                           Small             1.121986e+09
133175951034.47264


In [87]:
df['CurrentCompanyMarketCap'] = df['CurrentCompanyMarketCap'].fillna(df.groupby('CurrentCompanyMarketCapCategory')['CurrentCompanyMarketCap'].transform('mean'))

In [88]:
df['CurrentCompanyMarketCap'].fillna(totalMeanMarketCap, inplace = True)

In [89]:
df['Top12CMMCompany'].fillna('False', inplace = True)
df['Top24CMMCompany'].fillna('False', inplace = True)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27508 entries, 0 to 27527
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   LegacySFInstance                 27508 non-null  object        
 1   LegacyAccountName                27508 non-null  object        
 2   CurrentCompanyGroup              27508 non-null  object        
 3   CurrentCompanyMarketCap          27508 non-null  float64       
 4   CurrentCompanyMarketCapCategory  27508 non-null  object        
 5   Top12CMMCompany                  27508 non-null  object        
 6   Top24CMMCompany                  27508 non-null  object        
 7   OpportunityName                  27508 non-null  object        
 8   OpportunityType                  27508 non-null  object        
 9   TotalOpportunityAmount           26179 non-null  float64       
 10  CreateDate                       27508 non-null  datetime64[ns]

In [91]:
avgTotalOppAmt = df['TotalOpportunityAmount'].mean()
print(avgTotalOppAmt)
df['TotalOpportunityAmount'].fillna(avgTotalOppAmt, inplace = True)

748139.9005492954


In [92]:
CloseReasonMode = df['CloseReason'].mode()
print(CloseReasonMode)

df.loc[(df['CloseReason'].isna()) & (df['StageName'] == "Closed Lost"), 'CloseReason'] = "CloseReasonMode"

df.loc[(df['CloseReason'].isna()) & (df['StageName'] == "Closed Won"), 'CloseReason'] = "Won"


0    No Bid
Name: CloseReason, dtype: object


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27508 entries, 0 to 27527
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   LegacySFInstance                 27508 non-null  object        
 1   LegacyAccountName                27508 non-null  object        
 2   CurrentCompanyGroup              27508 non-null  object        
 3   CurrentCompanyMarketCap          27508 non-null  float64       
 4   CurrentCompanyMarketCapCategory  27508 non-null  object        
 5   Top12CMMCompany                  27508 non-null  object        
 6   Top24CMMCompany                  27508 non-null  object        
 7   OpportunityName                  27508 non-null  object        
 8   OpportunityType                  27508 non-null  object        
 9   TotalOpportunityAmount           27508 non-null  float64       
 10  CreateDate                       27508 non-null  datetime64[ns]

In [94]:
df['CreateDateMonth'] = df['CreateDate'].dt.month 

In [95]:
display(df)

LegacySFInstance                         LegacyAccountName  \
0                  RxC  Coram Speciality Infusion Services, Inc.   
1                  RxC                                 Caris-MPI   
2                  RxC                                Pharma Pac   
3                  RxC        Biologics Financial Reconciliation   
4                  RxC        Biologics Financial Reconciliation   
...                ...                                       ...   
27523              RHP                     Biocon Biologics Inc.   
27524              RHP                     Biocon Biologics Inc.   
27525              RHP                     Biocon Biologics Inc.   
27526              RHP                     Biocon Biologics Inc.   
27527              RHP                           Sequel Med Tech   

                            CurrentCompanyGroup  CurrentCompanyMarketCap  \
0      Coram Speciality Infusion Services, Inc.             8.923380e+10   
1                                     Caris-MPI             8.923380e+10   
2                                    Pharma Pac             8.923380e+10   
3            Biologics Financial Reconciliation             8.923380e+10   
4            Biologics Financial Reconciliation             8.923380e+10   
...                                         ...                      ...   
27523                     Biocon Biologics Inc.             1.331760e+11   
27524                     Biocon Biologics Inc.             1.331760e+11   
27525                     Biocon Biologics Inc.             1.331760e+11   
27526                     Biocon Biologics Inc.             1.331760e+11   
27527                           Sequel Med Tech             1.331760e+11   

      CurrentCompanyMarketCapCategory Top12CMMCompany Top24CMMCompany  \
0                               Large           False           False   
1                               Large           False           False   
2                               Large           False           False   
3                               Large           False           False   
4                               Large           False           False   
...                               ...             ...             ...   
27523                   Not Available           False           False   
27524                   Not Available           False           False   
27525                   Not Available           False           False   
27526                   Not Available           False           False   
27527                   Not Available           False           False   

                                         OpportunityName OpportunityType  \
0                                 Coram/CVS Copay Finder             New   
1                              Precision Medicine Survey             New   
2                                             Pharma Pac             New   
3            Biologics RARE - Launched for FY22 Tracking         Renewal   
4             Biologics ONC - Launched for FY22 Tracking         Renewal   
...                                                  ...             ...   
27523                            Biocon Biologics_TS SOW             New   
27524   Biocon Biologics_State Insulin Prog (MN, CO, ME)             New   
27525    Biocon Biologics_LS, TS & State Insulin Imp SOW             New   
27526  Biocon Biologics_State Insulin Prog (MN, CO, M...             New   
27527                  Sequel MedTech_MSA + eV & DC SOWs             New   

       TotalOpportunityAmount          CreateDate OpportunityOwner  \
0                7.481399e+05 2019-08-06 19:15:55    Andrew Burman   
1                2.000000e+05 2017-05-31 15:15:14  Jennifer Frytak   
2                1.738380e+05 2020-04-09 15:52:13    Ryan Grimmett   
3                4.498679e+07 2021-02-04 14:55:41      Laura Sieve   
4                1.808600e+07 2021-04-27 15:01:37      Laura Sieve   
...                       ...                 ...              ...

## Outliers
- Outliers can skew the results of data analysis and impact model performance
- Handling Outliers: 
    - Remove
    - Impute
    - Scale/Standardize

In [96]:
df.describe()

CurrentCompanyMarketCap  TotalOpportunityAmount  \
count             2.750800e+04            2.750800e+04   
mean              1.330849e+11            7.481399e+05   
min               6.493000e+01           -2.800000e+06   
25%               1.434807e+10            5.445000e+03   
50%               1.174317e+11            8.000000e+04   
75%               1.576153e+11            3.201000e+05   
max               6.998863e+11            1.188350e+09   
std               1.496705e+11            1.265793e+07   

                          CreateDate  CreateDateMonth  
count                          27508     27508.000000  
mean   2018-01-09 13:16:36.709248256         6.538316  
min              2008-03-17 18:16:52         1.000000  
25%              2015-06-19 15:57:31         4.000000  
50%              2018-09-17 22:26:58         7.000000  
75%       2021-03-01 18:16:57.500000         9.000000  
max              2024-02-19 20:48:25        12.000000  
std                              NaN         3.276847

In [101]:
df['CreateDateMonth'] = df['CreateDateMonth'].astype('object')
df['StageName'] = df['StageName'].astype('object')

In [122]:
df.describe(include='object')

LegacySFInstance  LegacyAccountName   CurrentCompanyGroup  \
count             27508              27508                 27508   
unique                3               1628                   919   
top                 RxC  Merck & Co., Inc.  Bristol Myers Squibb   
freq              15084                845                  1487   

       CurrentCompanyMarketCapCategory Top12CMMCompany Top24CMMCompany  \
count                            27508           27508           27508   
unique                               7               2               2   
top                              Large           False           False   
freq                             11747           17485           23058   

                          OpportunityName OpportunityType OpportunityOwner  \
count                               27508           27508            27508   
unique                              25829               3              249   
top     2016 Mid-America Cancer Symposium             New       Admin User   
freq                                   36           19518             2532   

         StageName CloseReason ImpactedFY  CreateDateMonth  
count        27508       27508      27508            27508  
unique           2          76         23               12  
top     Closed Won         Won    FY 2023                9  
freq         15376       13361       2838             3087

In [105]:
## Customize percentiles
df.describe(percentiles=[.01, .05, .1, .25, .5, .75, .90, .95, .99])  

CurrentCompanyMarketCap  TotalOpportunityAmount  \
count             2.750800e+04            2.750800e+04   
mean              1.330849e+11            7.481399e+05   
min               6.493000e+01           -2.800000e+06   
1%                9.376078e+05            0.000000e+00   
5%                1.477488e+08            0.000000e+00   
10%               1.164934e+09            0.000000e+00   
25%               1.434807e+10            5.445000e+03   
50%               1.174317e+11            8.000000e+04   
75%               1.576153e+11            3.201000e+05   
90%               3.055616e+11            8.022188e+05   
95%               3.799889e+11            1.955837e+06   
99%               6.998863e+11            8.973357e+06   
max               6.998863e+11            1.188350e+09   
std               1.496705e+11            1.265793e+07   

                          CreateDate  
count                          27508  
mean   2018-01-09 13:16:36.709248256  
min              2008-03-17 18:16:52  
1%        2008-05-19 20:03:47.160000  
5%     2010-03-10 04:13:26.450000128  
10%       2011-12-22 08:48:33.300000  
25%              2015-06-19 15:57:31  
50%              2018-09-17 22:26:58  
75%       2021-03-01 18:16:57.500000  
90%       2022-09-22 15:38:40.500000  
95%    2023-04-12 15:59:35.949999872  
99%    2023-11-06 19:02:27.330000128  
max              2024-02-19 20:48:25  
std                              NaN

In [106]:
import seaborn as sb
import statsmodels.api as sm

In [120]:
marketCapBox = sb.boxplot(x='CurrentCompanyMarketCap',data=df) 
#Decision: Use min max scalar to scale the data to remove the impactive outliers

## Exploratory Data Analysis
- Data Visualization

In [121]:
df['StageName'].value_counts()

StageName
Closed Won     15376
Closed Lost    12132
Name: count, dtype: int64

In [123]:
df['StageName'].value_counts().plot(kind='bar')

<Axes: xlabel='StageName'>

## Encoding
- Model cannot interpret strings, so we need to encode all the variables as numbers
- dummy variables
- Curse of dimensionality 

In [129]:
df['StageName'] = df['StageName'].map({'Closed Won': 1, 'Closed Lost': 0})

In [130]:
winRate = (sum(df['StageName'])/len(df['StageName'].index))*100
print(winRate)

55.89646648247783


In [131]:
df = pd.get_dummies(df, columns=["LegacySFInstance", "CurrentCompanyMarketCapCategory", "Top12CMMCompany", "Top24CMMCompany", "OpportunityType","ImpactedFY", "CreateDateMonth"], prefix=["LegacySFInstance", "CurrentCompanyMarketCapCategory", "Top12CMMCompany", "Top24CMMCompany", "OpportunityType","ImpactedFY", "CreateDateMonth"], drop_first=True)     

In [132]:
df.head()

LegacyAccountName  \
0  Coram Speciality Infusion Services, Inc.   
1                                 Caris-MPI   
2                                Pharma Pac   
3        Biologics Financial Reconciliation   
4        Biologics Financial Reconciliation   

                        CurrentCompanyGroup  CurrentCompanyMarketCap  \
0  Coram Speciality Infusion Services, Inc.             8.923380e+10   
1                                 Caris-MPI             8.923380e+10   
2                                Pharma Pac             8.923380e+10   
3        Biologics Financial Reconciliation             8.923380e+10   
4        Biologics Financial Reconciliation             8.923380e+10   

                               OpportunityName  TotalOpportunityAmount  \
0                       Coram/CVS Copay Finder            7.481399e+05   
1                    Precision Medicine Survey            2.000000e+05   
2                                   Pharma Pac            1.738380e+05   
3  Biologics RARE - Launched for FY22 Tracking            4.498679e+07   
4   Biologics ONC - Launched for FY22 Tracking            1.808600e+07   

           CreateDate OpportunityOwner  StageName         CloseReason  \
0 2019-08-06 19:15:55    Andrew Burman          0              No Bid   
1 2017-05-31 15:15:14  Jennifer Frytak          0  Client Funding Cut   
2 2020-04-09 15:52:13    Ryan Grimmett          0           Cancelled   
3 2021-02-04 14:55:41      Laura Sieve          1                 Won   
4 2021-04-27 15:01:37      Laura Sieve          1                 Won   

   LegacySFInstance_RHP  ...  CreateDateMonth_3  CreateDateMonth_4  \
0                 False  ...              False              False   
1                 False  ...              False              False   
2                 False  ...              False               True   
3                 False  ...              False              False   
4                 False  ...              False               True   

   CreateDateMonth_5  CreateDateMonth_6  CreateDateMonth_7  CreateDateMonth_8  \
0              False              False              False               True   
1               True              False              False              False   
2              False              False              False              False   
3              False              False              False              False   
4              False              False              False              False   

   CreateDateMonth_9  CreateDateMonth_10  CreateDateMonth_11  \
0              False               False               False   
1              False               False               False   
2              False               False               False   
3              False               False               False   
4              False               False               False   

   CreateDateMonth_12  
0               False  
1               False  
2               False  
3               False  
4               False  

[5 rows x 54 columns]